### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
import itertools
import re


%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

In [2]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
# reviews['timestamp'].mean()

In [4]:
pd.to_datetime('1362062307', unit='s')

Timestamp('2013-02-28 14:38:27')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [5]:
# # cell for work

# # 'The number of movies in the dataset'
# # cell for work

# # 'The number of ratings in the dataset':
# len(reviews)

# # more work
# # 'The number of different genres'

# len(movies['genre'].unique())

# # 'The number of unique users in the dataset':

# len(reviews['user_id'].unique())

# # 'The number missing ratings in the reviews dataset':
# reviews['rating'].isnull().sum()

# # 'The number missing ratings in the reviews dataset':
# reviews['rating'].mean()

# # 'The minimum rating given across all ratings':
# reviews['rating'].min()

# # 'The maximum rating given across all ratings':
# reviews['rating'].max()

# # and still more

# # and even more - open additional cells as necessary

# # # Use your findings to match each variable to the correct statement in the dictionary


# dict_sol1 = {
#     'The number of movies in the dataset': len(movies['movie'].unique()),
#     'The number of ratings in the dataset':
#     'The number of different genres':
#     'The number of unique users in the dataset':
#     'The number missing ratings in the reviews dataset':
#     'The average rating given across all ratings':
#     'The minimum rating given across all ratings':
#     'The maximum rating given across all ratings': }

# # # Originally, I had this to check your solution, but the
# # # links are live and updating.  That didn't end up being
# # # a great idea

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

### date and time

In [9]:
movies.dropna(inplace=True)

In [30]:
# Pull the date from the title and create new column

def create_year(x):
    try:
        year = re.findall("\(\d+\)", x)[0].strip('\()')
    except: 
        year = np.nan
    return year
    
movies['year'] = movies['movie'].apply(create_year)
movies['year'] = movies['year'].astype(int)

In [11]:
# Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)

movies['1800s'] = 0
movies['1900s'] = 0
movies['2000s'] = 0

for century, century_id in zip([18, 19, 20], ['1800s','1900s','2000s']):
    movies.loc[movies['year']//100 == century, century_id] = 1

### dummy movie genre with 1's and 0's

In [12]:
genres = [genre.split('|') for genre in movies['genre'].unique()]
merged = list(itertools.chain(*genres))
unique_genres = list(set(merged))

In [32]:
def dummy_movie_genre(genres):
    """Determine dummy genre for each movie, return with 1's and 0's """

    genre_indexes = [unique_genres.index(genre) for genre in genres]
    dummy_genre = len(unique_genres)*[0]

    for genre_index in genre_indexes:
        dummy_genre[genre_index] = 1

    return dummy_genre

In [33]:
dummy_genres = [] # empty list to store the dummy values
movie_genres = [genre.split('|') for genre in movies['genre']] # pull movie genres from the df

# Generate dummy genere for each movie
for movie_genre in movie_genres:
    dummy_genres.append(dummy_movie_genre(movie_genre))
    
# Update dummy values in the dataframe 
movies[unique_genres] = dummy_genres

### Create a date out of time stamp

In [15]:
reviews['timestamp'] = pd.to_datetime(reviews['timestamp'],errors = 'ignore', unit = 's')

In [17]:
# reviews['timestamp'].dt.year

In [38]:
movies['genre'].str.split('|', expand = True)

,0,1,2,3,4,5,6,7,8,9
0,Documentary,Short,None,None,None,None,None,None,None,None
1,Documentary,Short,None,None,None,None,None,None,None,None
2,Documentary,Short,None,None,None,None,None,None,None,None
4,Short,Horror,None,None,None,None,None,None,None,None
5,Short,Comedy,Horror,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
35473,Documentary,None,None,None,None,None,None,None,None,None
35474,Documentary,Adventure,Biography,Family,None,None,None,None,None,None
35475,Documentary,Music,None,None,None,None,None,None,None,None
35476,Documentary,None,None,None,None,None,None,None,None,None
